In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Еще раз про деревья решений

<img src='../imgs/tree_ex.gif' style="height:400px">

Формально, дерево решений - это связный ациклический граф. В нем можно выделить 3 типа вершин:
1. Корневая вершина (root node) -  откуда все начинается
2. Внутренние вершины (intermediate nodes)
3. Листья (leafs) - самые глубокие вершины дерева, в которых содержится "ответ"

Во внутренней или коневой вершине признак проверяется на некий логический критерий, по результатам которого мы движемся все глубже

## Меры неопределенности

Пусть $p_k$ - это доля класса $C_k$ в узле дерева $S$.

1. Missclassification error  
$$I(S) = 1 - \max\limits_k p_k $$
2. Gini index 
$$I(S) = 1 - \sum\limits_k (p_k)^2 = \sum\limits_{k'\neq k} p_{k'} p_k$$
3. Entropy 
$$I(S) = -\sum\limits_k p_k \log(p_k)$$



### Прирост информации

Выберем признак $A$ и пороговое значение $t$ на нем таким образом, чтобы уменьшить неопределенность:

**Насколько уменьшится неопределенность:** <br/>
$$ Gain(S, A) = I(S) - \left(\frac{|S_L|}{|S|}\cdot I(S_L) + \frac{|S_R|}{|S|}\cdot I(S_R) \right),$$ где $S_R$ и $S_L$ - это потомки узла $S$ c объектами, удовлетворяющим соответствующим условиям.

### Пример

<img src='../imgs/gain_ex.png'>

## Критерии останова (регуляризация)

* Никогда
* Задать порог по мере неопределенности: $I(S) \leq \theta$
* Задать порог по размеру узла: $|S| \leq n$
* Задать порог на глубину: $Depth(S) = d$
* Задать порог на размер потомков: $|S_L| \leq n_1 \& |S_R| \leq n_2$
* ...

## Регрессия

Для задачи регрессии в качестве меры неопределенности могут выступать

* Среднее квадратичное отклонение от среднего
$$ I(S) = \frac{1}{|S|}\sum\limits_{i \in S}(y_i - \bar{y_S})^2 $$
* Среднее абсолютное отклонение от среднего
$$ I(S) = \frac{1}{|S|}\sum\limits_{i \in S}|y_i - \bar{y_S}| $$

## Как определяется ответ?

* Классификация
    * Класс с большинством в листе
    * Доли каждого из классов в листе
* Регрессия
    * Среднее (медиана) целевой переменной в листе

## Важность признаков

В деревьях решений производится автоматический отбор признаков.

Пусть $v(S)$ - это признак, который использовался для ветвления в узле $S$

$$ \text{imp}(A) = \sum\limits_{i: v(S_i) = A} \frac{|S_i|}{|S|} Gain(S_i, A) $$

## Работа с пропусками

1. Удалить объекты\признаки с пропусками
2. Пропущенное значение = отдельная категория
3. Вычисление impurity без учета пропуска
4. Surrogate split

## Вычислительная сложность

#### Обучение
* Расчет мер неопределенности для $n$ объектов с $d$ признаками на одном уровне:
    * $O(dn)$
* Надо делать на каждом уровне дерева. Глубина дерева в сбалансированном случае случае - $\log{(n)}$
    * $O(dn \log{(n)})$
    
#### Применение
* В худшем случае $O(n)$
* В сбалансированном случае $O(\log{(n)})$

## Преимущества / Недостатки

** Преимущества **
* Простота построения
* Интерпретируемость (при небольшой глубине)
* Требуются минимальная предобработка признаков
* Встроенный отбор признаков




** Недостатки **
* Границы строяется только параллельно или перпендикулярно осям признаков
* При изменении набора данных надо полностью перестраивать и результат может получится совершенно иным
* Жадность построения

### Что нужно дополнительно понимать про деревья
* Любое недвоичное дерево решений можно представить двоичным
* Любая линейная комбинация деревьев решений есть дерево решений

# Ансамбли

Пусть нам даны базовы алгоритмы $a_1(x), a_2(x), ..., a_T(x)$, $a_i: \mathcal{X} \to \mathbb{R}$.

**Ансамблем** будем называть функцию 
$$h(x) = C(a_1(x), a_2(x), ..., a_T(x)),$$
$C: \mathbb{R}^T \to \mathcal{Y}$ -- решающее правило (метаалгоритм; то как будем комбинировать ответы базовых алгоритмов).

# Голосование

### Простое голосование

$$h(x) = \frac{1}{T} \sum_{i=1}^T a_i(x)$$

<img src="../imgs/vot1.png" style="height:400px">

### Взвешанное голосование

$$h(x) = \frac{1}{T} \sum_{i=1}^T b_i a_i(x)$$

<img src="../imgs/vot2.png" style="height:400px">

### Смесь экспертов

$$h(x) = \frac{1}{T} \sum_{i=1}^T b_i(x) a_i(x)$$

<img src="../imgs/vot3.png" style="height:400px">

# Blending

Обучаем базовые алгоритмы на части данных, потом их ответы на другой части данных используем как признаки для метаалгоритма.

<img src="../imgs/stacking.png" style="height:400px">

Какой недостаток у такого подхода?

# Stacking

Разбиваем обучающую выборку на фолды. Перебираем фолды, обучаем базовые алгоритмы на всех фолдах кроме текущего, на текущем делаем предсказания обученными алгоритмами. Полученные на каждом фолде предсказания используем как признаки для метаалгоритма.

Для предсказания на тесте, обучаем алгоритмы на всем обучающем множестве. Получаем метапризнаки на тесте, применив базовые алгоритмы к тестовым данным. Окончательный результат получаем применив метаалгоритм, обученный на обучающем множестве, к тестовым метапризнакам.

<img src="../imgs/stacking-2b.png" style="height:400px">

А есть ли недостатки у этого подхода?

Более подробнее про последние две техники [тут](https://dyakonov.org/2017/03/10/cтекинг-stacking-и-блендинг-blending/).

# Bias-variance decomposition

Будем рассматривать задачу регрессии, будем предпологать, что
$$y = f(x) + \epsilon,\,\,\,\, \epsilon \sim \mathcal{N}(0, \sigma).$$

Будем искать $h(x)$, приближающую исходную зависимость $f(x)$. Хотим оценить матожидание среднеквадратичной ошибки в некоторой точке $x_0$. Для этого будем сэмплировать $y$ из условного распределения $P(y|x=x_0)$.

$$err(x_0) = E_{P(y|x=x_0)}[(y - h(x_0))^2]$$

По простому это будет обычное MSE.

Вспомним определение дисперсии (вариации):
$$Var[y] = E[(y - E[y])^2] = E[y^2] - (E[y])^2.$$
Отсюда сразу
$$E[y^2] = Var[y] + (E[y])^2.$$

Также логично предположить, что изначальная зависимость $f(x)$ является детерминированной. Из этого следует, что
$$E[f(x_0)] = f(x_0).$$

Выражение для ошибки расписывается в
$$err(x_0) = E[y^2] + E[h^2(x_0)] - 2E[yh(x_0)].$$

Распишем отдельно члены последней суммы.

$$E[y^2] = Var[y] + (E[y])^2 = E[(y - E[y])^2] + (E[y])^2 = $$
$$= E[(\, f(x_0) + \epsilon - f(x_0) \,)^2] + f(x_0)^2 = E[\epsilon^2] + f(x_0)^2 =$$
$$=Var[\epsilon] + (E[\epsilon])^2 + f(x_0)^2 = Var[\epsilon] + f(x_0)^2$$

$$E[h^2(x_0)] = Var[h(x_0)] + (E[h(x_0)])^2$$

$$E[yh(x_0)] = E[f(x_0)h(x_0) + \epsilon h(x_0)] = f(x_0)E[h(x_0)]$$

$$err(x_0) = Var[\epsilon] + \underline{f(x_0)^2} + Var[h(x_0)] + \underline{(E[h(x_0)])^2} - \underline{2f(x_0)E[h(x_0)]} = $$
$$= Var[\epsilon] + Var[h(x_0)] + (f(x_0) - E[h(x_0)])^2 = $$
$$= (f(x_0) - E[h(x_0)])^2 + E[(h(x_0) - E[h(x_0)])^2] + Var[\epsilon] =$$
$$= \text{Bias}^2 + \text{Variance} + \text{Noise}$$

<table><tr>
<td><img src="../imgs/bias-var1.png" style="height:400px"></td>
<td><img src="../imgs/bias-var2.png" style="height:400px"></td>
</tr></table>

# Random forest

Возмем в качестве базовых алгоритмов решающие деревья. Обучим каждое, а ответы их усредним (простое голосование).

Что из этого получится?

### Небольшой эксперимент
* Какая высота у Эйфелевой башни? (без гугла :))

Стратегии

* Bagging = Bootstrap aggregation. Обучаем алгоритмы по случайным подвыборкам размера N, полученным с помощью выбора с возвращением.
* RSM = Random Subspace Method. Метод случайных подпространств. Обучаем алгоритмы по случайным подмножествам признаков.

Таким образом Random Forest = Деревья + Bagging + RSM

In [2]:
from sklearn.ensemble import RandomForestClassifier
from ipywidgets import interact, IntSlider
from sklearn.datasets import make_moons

def rf_demo(n_est=5):
    rf = RandomForestClassifier(random_state=123, n_estimators=n_est)

    np.random.seed(0)
    
    X, y = make_moons(noise=0.3, random_state=123)
    rf.fit(X, y)
    
    x_range = np.linspace(X.min(), X.max(), 100)
    xx1, xx2 = np.meshgrid(x_range, x_range)
    plt.figure(figsize=(10,8))
    
    for tree in rf.estimators_:
        y_hat = tree.predict(np.c_[xx1.ravel(), xx2.ravel()])
        y_hat = y_hat.reshape(xx1.shape)

        plt.contourf(xx1, xx2, y_hat, alpha=1.0/n_est)
    plt.scatter(X[:,0], X[:,1], c=y)
    
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    
    plt.title('N estimators = %d' % n_est)
    plt.show()

In [3]:
try:
    fig = interact(rf_demo, n_est=IntSlider(min=1, max=101, value=1, step=5))
except:
    print('Что-то не так. Посмотрите на доску')

interactive(children=(IntSlider(value=1, description='n_est', max=101, min=1, step=5), Output()), _dom_classes…

## Bias-variance для случайного леса

$$h(x) = \frac{1}{T}\sum_{i=1}^T a_i(x),$$
$$Var[a_i(x_0)] = \sigma^2, \,\, E[a_i(x_0)] = \mu.$$

### Bias

$$\text{Bias}^2 = (E[h(x_0)] - f(x_0))^2 = (E[a(x_0)] - f(x_0))^2$$

### Variance

$$\text{Variance} = E[(h(x_0) - E[h(x_0)])^2] =$$
$$= \frac{1}{T^2}\sum_{i=1}^T \sum_{j=1}^T E\left[\,(a_i(x_0) - E[a_i(x_0)])\,(a_j(x_0) - E[a_j(x_0)])\,\right]=$$
$$=\frac{1}{T^2}\sum_{i=1}^T\left(\sigma^2 + \sum_{j \neq i}\text{cov}(a_i(x_0), a_j(x_0))\right) = $$
$$=\frac{1}{T^2}\sum_{i=1}^T \left(\sigma^2 + (T-1)\sigma^2\rho \right)=$$
$$=\frac{\sigma^2}{T} + \frac{(T-1)\sigma^2 \rho}{T} = \rho \sigma^2 + \sigma^2\frac{1-\rho}{T}$$

Случайный лес не переобучается с ростом $T$.

# Gradient boosting

Используем модель взвешанного голосования:
$$h(x) = \frac{1}{T} \sum_{i=1}^T b_i a_i(x)$$

Добавляем новый алгоритм к нашей композиции, минимизируя следующую ошибку:
$$err(h) = \sum_{j=1}^N L\left(y_i, \sum_{i=1}^{T-1}b_i a_i(x_j) + b\cdot a(x_j)\right) \to \min_{a,b}$$

Ключевая идея в том, чтобы искать очередной $a_i$, который будет аппроксимировать антиградиент ошибки $err(h_{i-1})$ по параметрам $[h_{i-1}(x_1), ..., h_{i-1}(x_N)]$:
$$h_{i}(x_j) = h_{i-1}(x_j) - \eta \cdot {err}'(h_{i-1}(x_j)).$$

## Алгоритм обучения GB

1. Инициализируем $h_0(x) = \arg\min_{\gamma}\sum_{j=1}^N L(y_i, \gamma)$;

2. Для все $i$ от 1 до T:
    * Для всех $j=1,...,N$ вычисляем
    $$g_{i,j} = -\left[\frac{\partial L(y_i, h(x_j))}{\partial h(x_j)}\right]_{h=h_{i-1}};$$
    * Строим базовую модель $a_i(x)$, беря в качестве таргета $g_{i,j}$:
    $$a_i = \arg\min_a \sum_{j=1}^N (g_{i,j} - a(x_j)^2;$$
    * Определить вес $b_i$:
    $$b_i = \arg\min_b \sum_{j=1}^N L\left(y_i, h_{i-1}(x_j) + b\cdot a_i(x_j)\right);$$
    * Обновить модель:
    $$h_i(x) = h_{i-1}(x) + b_i \cdot a_i(x);$$
    
3. Возращаем $h_T(x)$.

### Факты о бустинге
* В отличии от бэггинга (RF), который уменьшает только дисперсию модели, бустинг уменьшает также и смещение.
* Градиентный бустинг может переобучаться с ростом $T$.

<img src="../imgs/overfit.png" style="height:400px">

Побаловаться: http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html

# Регуляризация
* Можно добавлять алгоритмы с некоторым дисконтом (аля скорость спуска $\eta$).
* Использовать Bagging.
* Что еще можно придумать?

### Сложность
* Вычисление градиента O (g (N ))
* Построение модели (дерева) O (HND )
* Вычисление предсказания O (N )

где H - высота дерева.

Построение дерева может быть эффективно распараллелено по фичам. Поэтому в целом быстро.